# Подготовка файлов и программ

Скачиваем файл scaffolds.fasta из первого дз 

In [ ]:
!wget -nc https://raw.githubusercontent.com/greggasd/hse21_hw1/main/data/scaffolds.fasta

--2021-12-18 10:47:02--  https://raw.githubusercontent.com/greggasd/hse21_hw1/main/data/scaffolds.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3961243 (3.8M) [text/plain]
Saving to: ‘scaffolds.fasta’

scaffolds.fasta     100%[===================>]   3.78M  --.-KB/s    in 0.07s   

2021-12-18 10:47:02 (53.4 MB/s) - ‘scaffolds.fasta’ saved [3961243/3961243]



Скачиваем GeneMark и ключ от него
http://opal.biology.gatech.edu/GeneMark/license_download.cgi

Скачиваем по ссылке и кидаем в директорию колаба

In [ ]:
!gzip -d gm_key_64.gz
!tar -xzvf gms2_linux_64.tar.gz
!cp -v gm_key_64 ~/.gmhmmp2_key

gms2_linux_64/
gms2_linux_64/gms2.pl
gms2_linux_64/README_GMS2.txt
gms2_linux_64/gmhmmp2_key
gms2_linux_64/mgm_4.mod
gms2_linux_64/gmhmmp2
gms2_linux_64/INSTALL
gms2_linux_64/mgm_11.mod
gms2_linux_64/biogem
gms2_linux_64/LICENSE
gms2_linux_64/mgm_25.mod
gms2_linux_64/compp
gms2_linux_64/mgm_15.mod
'gm_key_64' -> '/root/.gmhmmp2_key'


Качаем данные по бактерии T.oleivorans

In [ ]:
!curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh | sh -
!~/edirect/efetch -db nuccore -id HF680312 -format gb > T_oleivorans_MIL_1.gbk
!~/edirect/efetch -db nuccore -id HF680312 -format gene_fasta > T_oleivorans_MIL_1.genes.fasta
!~/edirect/efetch -db nuccore -id HF680312 -format fasta_cds_aa > T_oleivorans_MIL_1.proteins.fasta


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
Holding off, then.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



Добавляем остальные прогрмаммы

In [ ]:
!apt update
!apt install ncbi-blast+
!wget https://github.com/shenwei356/seqkit/releases/download/v2.1.0/seqkit_linux_amd64.tar.gz
!tar -xzvf seqkit_linux_amd64.tar.gz
!chmod a+x seqkit

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/

Скачиваем БД белков SwissProt

In [ ]:
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz
!gzip -d uniprot_sprot.fasta.gz

--2021-12-18 10:58:30--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90527596 (86M) [application/x-gzip]
Saving to: ‘uniprot_sprot.fasta.gz’

uniprot_sprot.fasta 100%[===================>]  86.33M  74.0MB/s    in 1.2s    

2021-12-18 10:58:31 (74.0 MB/s) - ‘uniprot_sprot.fasta.gz’ saved [90527596/90527596]



# Выполнение первого задания

Первым делом ищем гены

In [ ]:
!./gms2_linux_64/gms2.pl  --seq scaffolds.fasta  --genome-type bacteria  --fnn genes.fasta  --faa proteins.fasta

Далее определяем их назначение

In [ ]:
!makeblastdb -dbtype prot -in T_oleivorans_MIL_1.proteins.fasta -out T_oleivorans_MIL_1.proteins
!blastp -query proteins.fasta -db T_oleivorans_MIL_1.proteins -evalue 1e-10 -outfmt 6 > scaffolds.hits_from_MIL_1.txt


!cut -f 1 scaffolds.hits_from_MIL_1.txt | sort -n | uniq > proteins.with_hits_from_MIL_1.txt
!./seqkit grep --invert-match -f proteins.with_hits_from_MIL_1.txt proteins.fasta -o proteins.without_MIL_1.fasta


!makeblastdb -dbtype prot -in uniprot_sprot.fasta -out uniprot_sprot
!blastp -query proteins.without_MIL_1.fasta -db uniprot_sprot -evalue 1e-10 -outfmt 6 > scaffolds.hits_from_SwissProt.txt



Building a new DB, current time: 12/18/2021 11:01:08
New DB name:   /content/T_oleivorans_MIL_1.proteins
New DB title:  T_oleivorans_MIL_1.proteins.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 3666 sequences in 0.098691 seconds.
[INFO] 3328 patterns loaded from file


Building a new DB, current time: 12/18/2021 11:02:11
New DB name:   /content/uniprot_sprot
New DB title:  uniprot_sprot.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 565928 sequences in 12.9599 seconds.


И теперь считаем финальную статистику

In [ ]:
total_genes = !grep -c '>' genes.fasta
total_genes = int(total_genes[0])

rest_genes = !grep -c '>' proteins.without_MIL_1.fasta
rest_genes = int(rest_genes[0])

swiss_genes = !cut -f 1 scaffolds.hits_from_SwissProt.txt | sort | uniq | wc -l
swiss_genes = int(swiss_genes[0])

print("Cколько было предсказано генов всего?", total_genes)
print("Cколько из них удалось аннотировать с помощью сравнения с бактерией?", total_genes-rest_genes)
print("Сколько с помощью БД SwissProt?", swiss_genes)
print("Kакое кол-во белков остались без аннотации функции?", rest_genes-swiss_genes)

Cколько было предсказано генов всего? 3606
Cколько из них удалось аннотировать с помощью сравнения с бактерией? 3328
Сколько с помощью БД SwissProt? 52
Kакое кол-во белков остались без аннотации функции? 226
